In [ ]:
!npm install -g localtunnel
!sudo apt-get install xvfb xserver-xephyr tigervnc-standalone-server x11-utils gnumeric novnc
!apt-get update
!apt-get install -y wget unzip x11vnc
!sudo apt update
!sudo apt install nginx
!pip install pyvirtualdisplay pillow EasyProcess websockets
# Install Chrome
!wget -q https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!dpkg -i google-chrome-stable_current_amd64.deb
!npm install -g localtunnel
!apt --fix-broken install -y
from IPython.display import clear_output
import os
clear_output()

In [ ]:
from easyprocess import EasyProcess
from pyvirtualdisplay import Display
from pyvirtualdisplay.smartdisplay import SmartDisplay
import threading , time , os , socket ,threading , requests , threading
from IPython.display import HTML , IFrame


def get_public_ip():
    response = requests.get('https://api.ipify.org?format=json')
    return response.json()['ip']
def get_available_port():
    # Create a socket using the IPv4 address family and TCP protocol
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

    # Bind to a port of value 0 which allows the OS to assign an available port
    sock.bind(('localhost', 0))

    # Retrieve the port number assigned by the OS
    port = sock.getsockname()[1]

    # Close the socket
    sock.close()

    return port

In [ ]:
!port_to_check=5900; pid=$(lsof -t -i:$port_to_check); if [ -n "$pid" ]; then echo "Process using port $port_to_check found with PID: $pid"; kill $pid; echo "Process with PID $pid terminated."; else echo "No process found using port $port_to_check."; fi

def background_task():
    with SmartDisplay(backend="xvnc", size=(1000, 600)) as disp:
        os.environ['DISPLAY'] = f":{str(disp.display )}"
        with EasyProcess(["google-chrome" , "--no-sandbox" ,  "--app-url" , " https://www.google.com/" ])  as proc:
            time.sleep(5)
            img = disp.waitgrab()
            print(disp.display , file = open('debug.txt' , 'w'))
            #### save this image to local
            img.save('screenshot.png')  # Save the image to local file
            proc.wait()
            print("disp.display" , file = open('debug.txt' , 'w'))
# Start the background task in a separate thread

background_thread = threading.Thread(target=background_task)
background_thread.daemon = True
background_thread.start()
!rm nohup.out
!nohup x11vnc -quiet -cursor  -bg -localhost -nopw -forever -shared -rfbport 5904 --multiptr  -create &
!port_to_check=6080; pid=$(lsof -t -i:$port_to_check); if [ -n "$pid" ]; then echo "Process using port $port_to_check found with PID: $pid"; kill $pid; echo "Process with PID $pid terminated."; else echo "No process found using port $port_to_check."; fi
!nohup /usr/share/novnc/utils/launch.sh --vnc localhost:5904 > novnc.log 2>&1 &
time.sleep(5)
url_novnc = None
with open('novnc.log') as buff:
    for line in buff :
        if line.strip().startswith('http://') :
            url_novnc = line.strip()
            print(url_novnc)


In [ ]:
from flask import Flask, request
import asyncio
import websockets
!port_to_check=5000; pid=$(lsof -t -i:$port_to_check); if [ -n "$pid" ]; then echo "Process using port $port_to_check found with PID: $pid"; kill $pid; echo "Process with PID $pid terminated."; else echo "No process found using port $port_to_check."; fi

app = Flask(__name__)
local_server_uri = url_novnc
print(local_server_uri)

@app.route('/')
async def websocket():
     # Replace with your local WebSocket server URI
    ws = await websockets.connect(local_server_uri)

    async def forward_request():
        async for message in request.ws:
            await ws.send(message)

    async def forward_response():
        async for message in ws:
            await request.ws.send(message)

    await asyncio.gather(forward_request(), forward_response())
    return ''

def run_flask() :
    app.run(host='0.0.0.0', port=5000)
background_thread2 = threading.Thread(target=run_flask)
background_thread2.daemon = True
background_thread2.start()

In [ ]:
available_port = get_available_port()
os.environ["AVAILABLEPORT"] = str(available_port)
!nohup lt --port 5000 > localtunnel.log 2>&1 &
public_ip = get_public_ip()

html_content = f"""

<div style="width: 100% , padding:12 ; display: flex;justify-content: center; ">
    <button id="copyButton" onclick="copyToClipboard()" style="width: 100%;margin: 12px ; padding: 12px">Copy Ip address</button>
    <script>
        function copyToClipboard() {{

            navigator.clipboard.writeText("{public_ip}") ;
        }}
    </script>
</div>
"""
display(HTML(html_content))
loacl_tunnel_url = open("localtunnel.log").read().strip().split()[-1].strip()
IFrame(loacl_tunnel_url,'100%','900')

In [ ]:
!port_to_check=5900; pid=$(lsof -t -i:$port_to_check); if [ -n "$pid" ]; then echo "Process using port $port_to_check found with PID: $pid"; kill $pid; echo "Process with PID $pid terminated."; else echo "No process found using port $port_to_check."; fi

def background_task2():
    with SmartDisplay(backend="xvnc", size=(1000, 600),rfbport=5904) as disp:
        os.environ['DISPLAY'] = f":{str(disp.display )}"
        with EasyProcess(["google-chrome" , "--no-sandbox" ,  "--app-url" , "http://a6b21e7dacbd:6080/vnc.html?host=a6b21e7dacbd&port=6080" ])  as proc:
            time.sleep(5)
            img = disp.waitgrab()
            print(disp.display , file = open('debug.txt' , 'w'))
            #### save this image to local
            img.save('screenshot2.png')  # Save the image to local file
            proc.wait()
            print("disp.display" , file = open('debug.txt' , 'w'))

# Start the background task in a separate thread

background_thread3 = threading.Thread(target=background_task2)
background_thread3.daemon = True
background_thread3.start()

In [ ]:
loacl_tunnel_url = open("localtunnel.log").read().split()[-1].strip()
loacl_tunnel_url